In [1]:
from IPython.core.display import display, HTML
import sys
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
from datetime import date, datetime
import numpy as np
import pandas as pd
import dataframe_image as dfi
# pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
#pd.set_option('display.max_rows', 300)
#pd.set_option('display.max_columns', 20)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger, Fetcher
import WUBRG
from WUBRG import get_color_identity
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils import get_next_17lands_update_time, get_prev_17lands_update_time, get_name_slice, get_color_slice, get_date_slice


TRGT_SET = 'DMU'
LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

# Initialization

In [3]:
data_manager = None
set_data = None
print(f'Available sets: {SETS}')
print(f'Target set: {TRGT_SET}')

Available sets: ['DMU', 'SNC', 'NEO', 'VOW', 'MID']
Target set: DMU


In [4]:
print(f"Current Local Time:  {datetime.now()}")
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Current Local Time:  2022-08-31 00:26:33.405036
Last 17Lands Update: 2022-08-31 02:00:00
Current UTC Time:    2022-08-31 02:56:33.405036
Next 17Lands Update: 2022-09-01 02:00:00


In [5]:
start = datetime.utcnow()

if set_data is None:
    set_data = SetManager(TRGT_SET)
set_data.check_for_updates()
card_dict = set_data.SET_METADATA.CARD_DICT

end = datetime.utcnow()
print(f"\n --- Data loaded in {end - start}.")

Loading set metadata for: DMU
Fetching data for set: DMU
Fetching card data for set: DMU
Done!

Checking for missing data for DMU PremierDraft...
DMU PremierDraft has no summary data to get!
DMU PremierDraft has no historic data to get!
Finished checking for missing data for DMU PremierDraft.

Checking for missing data for DMU TradDraft...
DMU TradDraft has no summary data to get!
DMU TradDraft has no historic data to get!
Finished checking for missing data for DMU TradDraft.

Checking for missing data for DMU QuickDraft...
DMU QuickDraft has no summary data to get!
DMU QuickDraft has no historic data to get!
Finished checking for missing data for DMU QuickDraft.


 --- Data loaded in 0:00:28.123774.


## Tier List Analysis

### Functions

In [6]:
tier_to_rank = {
    "A+": 12,
    "A": 11,
    "A-": 10,
    "B+": 9,
    "B": 8,
    "B-": 7,
    "C+": 6,
    "C": 5,
    "C-": 4,
    "D+": 3,
    "D": 2,
    "D-": 1,
    "F": 0,
    "SB": None,
    "TBD": None
}

rank_to_tier = {v: k for k, v in tier_to_rank.items()}

In [7]:
def fetch_raw_data(url):
    if url.startswith("https://www.17lands.com/tier_list/"):
        guid = url.replace("https://www.17lands.com/tier_list/", "")
        url = f"https://www.17lands.com/card_tiers/data/{guid}"
    fetcher = Fetcher()
    raw_data = fetcher.fetch(url)
    return raw_data


def parse_raw_data(raw_data):
    data = dict()
    for card_rating in raw_data:
        data[card_rating['name']] = {
            'card': card_dict[card_rating['name']], 
            'tier': card_rating['tier'], 
            'rank': tier_to_rank[card_rating['tier']], 
            'synergy': card_rating['flags']['synergy'], 
            'buildaround': card_rating['flags']['buildaround']
        }
    return data


def to_data_frame(data, col_name='Rank'):
    ranks = dict()
    for card in data:
        ranks[card] = data[card]['rank']

    return pd.DataFrame.from_dict(ranks, orient="index", columns=[col_name])


def frame_from_url(url, name):
    raw_data = fetch_raw_data(url)
    data = parse_raw_data(raw_data)
    frame = to_data_frame(data)
    frame.index.name = name
    
    return frame

In [8]:
def merge_rankings(frame_list):
    frame = frame_list[0].copy(True)
    names = list()
    
    for indiv in frame_list:
        name = indiv.index.name
        names.append(name)
        frame[name] = indiv['Rank'].astype('Int64')
        
    frame = frame.drop('Rank', axis=1)
    frame['mean'] = frame.mean(axis=1)
    frame['max'] = frame.max(axis=1)
    frame['min'] = frame.min(axis=1)
    frame['range'] = frame['max'] - frame['min']
    frame['range'] = frame['range']
    frame.index.name = 'Card'
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(frame['mean'] - frame[name])
    dist['dist'] = dist.mean(axis=1)
    frame['dist'] = dist['dist'].round(1)

    return frame


def supplement_frame(frame, card_dict):
    links = dict()
    cast = dict()
    colors = dict()
    rarities = dict()
    cmc = dict()
    
    for card_name in card_dict:
        card = card_dict[card_name]
        links[card_name] = card.NAME
        cast[card_name] = card.CAST_IDENTITY
        colors[card_name] = card.COLOR_IDENTITY
        rarities[card_name] = card.RARITY
        cmc[card_name] = card.CMC
    
    series = frame.index.to_series()
    frame['Image'] = series.map(links)
    frame['To Cast'] = series.map(cast)
    frame['Identity'] = series.map(colors)
    frame['Rarity'] = series.map(rarities)
    frame['CMC'] = series.map(cmc)
    
    cols = list(frame.columns)
    frame = frame[['Image', 'CMC', 'Rarity', 'Identity', 'To Cast'] + cols[:-5]]

    return frame

In [9]:
from enum import Flag, auto

class ColorSortStyles(Flag):
    exact = auto()
    subset = auto()
    contains = auto()
    superset = contains
    adjascent = auto()
    shares = auto()
    
    
def _parse_to_set(val, var_name):
    t = type(val)
    if t is str:
        val = set(val)
    elif t is list:
        val = set(val)
    elif t is set:
        pass
    else:
        raise TypeError(f"Invalid type for `{var_name}`. Use `str`, `list`, or `set`.")
        
    ret = set()
    for i in val:
        ret.add(i.upper())
    return ret


def rarity_filter(rarities):
    rarities = _parse_to_set(rarities, 'rarities')
    allowed = set('CURM')
    
    if len(rarities - allowed) != 0:
        raise ValueError(f"Set must be composed of subset of {allowed}")
    
    return lambda frame : frame['Rarity'].isin(rarities)


def cmc_filter(cmc, op):
    if type(cmc) is not int:
        raise ValueError("`cmc` must be an int.")
    
    ops = {
        '>' : lambda frame : frame['CMC'] > cmc,
        '<' : lambda frame : frame['CMC'] < cmc,
        '==' : lambda frame : frame['CMC'] == cmc,
        '!=' : lambda frame : frame['CMC'] != cmc,
        '>=' : lambda frame : frame['CMC'] >= cmc,
        '<=' : lambda frame : frame['CMC'] <= cmc
    }
    
    if op not in ops:
        raise ValueError(f"`op` must be one of {ops}")
    
    return ops[op]


def color_filter(colors, style, col_name):
    colors = _parse_to_set(colors, 'colors')
    allowed = set('WUBRG')
    
    if len(colors - allowed) != 0:
        raise ValueError(f"Set must be composed of subset of {allowed}")
        
    colors = get_color_identity(''.join(colors))
    
    # Does the card colour exactly match the colour filter
    def _exact(frame):
        return frame[col_name] == colors
    
    # Is the card colour a non-strict subset of the colour filter
    def _subset(frame):
        return frame[col_name].isin(WUBRG.get_color_subsets(colors))
    
    # Is the card colour a non-strict superset of the colour filter
    def _superset(frame):
        return frame[col_name].isin(WUBRG.get_color_supersets(colors))
    
    # Is the card colour no more than one colour different of the colour filter
    def _adjascent(frame):
        subset = WUBRG.get_color_subsets(col, len(col) - 1, True)
        superset = WUBRG.get_color_supersets(col, len(col) + 1)
        return frame[col_name].isin(subset + superset)
    
    # Does the card colour share any colour with the colour filter
    def _shares(frame):
        shared = set()
        for color in colors:
            shared = shared.union(set(WUBRG.get_color_supersets(color)))
        return frame[col_name].isin(shared)
    
    funcs = {
        ColorSortStyles.exact : _exact,
        ColorSortStyles.subset : _subset,
        ColorSortStyles.contains : _superset,
        ColorSortStyles.superset : _superset,
        ColorSortStyles.adjascent : _adjascent,
        ColorSortStyles.shares : _shares
    }
    
    if style not in funcs:
        raise ValueError(f"`style` must be one of `ColorSortStyles` enums")
    
    return funcs[style]


def style_frame(sub_frame):
    def hoverable(card_name):
        card = card_dict[card_name]
        hmtl = '<style>.hover_img a { position:relative; }\n' + \
        '.hover_img a span { position:absolute; display:none; z-index:300; }\n' + \
        '.hover_img a:hover span { display:block; height: 300px; width: 300px; overflow: visible; margin-left: -175px; }</style>\n' + \
        f'<div class="hover_img">\n' + \
        f'<a href="#">{card_name}<span><img src="{card.IMAGE_URL}" alt="image"/></span></a>\n' + \
        f'</div>'
        return hmtl
     
    def format_short_float(val):
        return '{:.1f}'.format(val)
    
    def format_long_float(val):
        return '{:.3f}'.format(val)
    
    sub_frame = sub_frame.style.format({
        'Image': hoverable,
        'mean': format_long_float,
        'max': format_short_float,
        'min': format_short_float,
        'range': format_short_float,
        'dist': format_short_float,
    })
    
    return sub_frame


def display_frame(frame, order='mean', filters=None):
    pd.set_option('display.max_rows', 300)
    sub_frame = frame
    
    if filters is not None:
        filt_frame = pd.DataFrame()
        for f in filters:
            filt_frame[id(f)] = f(sub_frame)
        sub_frame = sub_frame[filt_frame.T.all()]
            
    sub_frame = sub_frame.sort_values(order, ascending=False)
            
    return style_frame(sub_frame)

In [10]:
def get_avgs(frame):
    colors = 'WUBRG'
    rarities = 'MRUC'
    avgs = dict()
    
    for r in rarities:
        color_frame = pd.DataFrame()
        
        for c in colors:    
            filt_frame = pd.DataFrame()
            filt_frame['color'] = color_filter(c, ColorSortStyles.exact, 'To Cast')(frame)
            filt_frame['rarity'] = rarity_filter(r)(frame)
            working = frame[filt_frame.T.all()].copy(True)
            working = working.drop(['min', 'max', 'range', 'dist', 'Image', 'CMC', 'Rarity', 'Identity', 'To Cast'], axis=1)
            color_frame[c] = working.mean().round(1)
            
        avgs[r] = color_frame.T
    
    ret = pd.concat(avgs)
    names = list(ret.columns)[:-1]
    
    ret['max'] = ret.max(axis=1)
    ret['min'] = ret.min(axis=1)
    ret['range'] = ret['max'] - ret['min']
    
    # Get the difference of squares distance to figure out most 'controversial' cards.
    dist = pd.DataFrame()
    for name in names:
        dist[name] = abs(ret['mean'] - ret[name])
    dist['dist'] = dist.mean(axis=1)
    ret['dist'] = dist['dist'].round(2)

    return ret

### Data Analysis

After loading all of the data, we crunch it to figure out the average grade that each contributer has given to cards. While not totally necessary, this helps anchor evaluations between contrinutors, as one person's C may be another person's C+. 

In [11]:
pairs = [
    ("https://www.17lands.com/tier_list/03ab10d25d8841f8aef0aa90b30c434b", 'Zac'),
    ("https://www.17lands.com/tier_list/3078f70b9a0d415ebf9f555439f5aedb", 'Klarm'),
    ("https://www.17lands.com/tier_list/a1ae9a695abb4921866f7f4a9a22e21f", 'Phyrre'),
    ("https://www.17lands.com/tier_list/19ee3198a63b499bb7c25e6ceaae00ba", 'Nomad'),
    ("https://www.17lands.com/tier_list/f1b015031fd741268c5228d4d8435938", 'Catharsis'),
    ("https://www.17lands.com/tier_list/e854e4a17f5147ad9908c5f6e5d2e87c", "Rewind"),
    ("https://www.17lands.com/tier_list/1d343c3e1f0d4cfd8c356cdaa42bb329", "Davis")
]

indiv_frames = [frame_from_url(*pair) for pair in pairs]
base_frame = merge_rankings(indiv_frames)
frame = supplement_frame(base_frame.copy(True), card_dict)
base_frame.mean()

Zac          5.095785
Klarm        5.544402
Phyrre       5.084291
Nomad        5.855469
Catharsis    5.100775
Rewind       5.678295
Davis        5.443580
mean         5.369239
max          6.946360
min          3.804598
range        3.141762
dist         0.914176
dtype: float64

### 'Controversial' Cards

Here, we separate the cards by rarity, and order them by the sum of the difference between each rating and the mean. This is saved in the `dist` column, and the higher the `dist` the lesss people agree on the rating of a card. If the `dist` is zero, everyone has given the card the same grade. The value of dist can either mean that some people disagree and most agree, or everyone disagrees to varying degrees.

To me, anyting with a dist less than or equal to 0.6 is pretty agreed upon, and anything higher than 1.0 is fairly contentious.

In [12]:
ordering = 'dist'
commons = display_frame(frame, order=ordering, filters=[rarity_filter('C')]).data
uncommons = display_frame(frame, order=ordering, filters=[rarity_filter('U')]).data
rare = display_frame(frame, order=ordering, filters=[rarity_filter('R')]).data
mythic = display_frame(frame, order=ordering, filters=[rarity_filter('M')]).data

display = pd.concat([commons, uncommons, rare, mythic])
style_frame(display)

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist
Card,,,,,,,,,,,,,,,,,
Crystal Grotto,Crystal Grotto,0,C,,,0,1,3,4,5,6,1,2.857,6.0,0.0,6.0,1.9
Toxic Abomination,Toxic Abomination,2,C,B,B,1,5,2,1,4,4,5,3.143,5.0,1.0,4.0,1.6
Deathbloom Gardener,Deathbloom Gardener,3,C,G,G,3,6,6,2,2,3,5,3.857,6.0,2.0,4.0,1.6
Vanquisher's Axe,Vanquisher's Axe,1,C,,,1,4,2,1,4,4,1,2.429,4.0,1.0,3.0,1.3
Academy Wall,Academy Wall,3,C,U,U,4,4,4,2,2,1,5,3.143,5.0,1.0,4.0,1.3
Writhing Necromass,Writhing Necromass,7,C,B,B,3,4,3,6,5,3,6,4.286,6.0,3.0,3.0,1.2
Salvaged Manaworker,Salvaged Manaworker,2,C,,,4,2,4,5,2,6,3,3.714,6.0,2.0,4.0,1.2
Eerie Soultender,Eerie Soultender,3,C,B,B,1,2,5,4,3,4,5,3.429,5.0,1.0,4.0,1.2
Colossal Growth,Colossal Growth,2,C,RG,G,3,6,5,6,3,4,5,4.571,6.0,3.0,3.0,1.1


### Top Cards

This is the short-list of top commons and uncommons, ordered by colour, based on average rating of contributors.

In [13]:
ordering = 'mean'
top = list()

for c in 'WUBRG':
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('U'), color_filter(c, ColorSortStyles.exact, 'To Cast')]).data.head(3))
    top.append(display_frame(frame, order=ordering, filters=[rarity_filter('C'), color_filter(c, ColorSortStyles.exact, 'To Cast')]).data.head(5))

display = pd.concat(top)
style_frame(display)

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist
Card,,,,,,,,,,,,,,,,,
Phyrexian Missionary,Phyrexian Missionary,2,U,WB,W,7,8,7,9,8,9,6,7.714,9.0,6.0,3.0,0.9
Prayer of Binding,Prayer of Binding,4,U,W,W,7,8,8,8,7,7,7,7.429,8.0,7.0,1.0,0.5
Knight of Dawn's Light,Knight of Dawn's Light,2,U,W,W,7,7,7,9,7,7,6,7.143,9.0,6.0,3.0,0.5
Argivian Cavalier,Argivian Cavalier,3,C,W,W,7,7,7,8,7,6,4,6.571,8.0,4.0,4.0,0.9
Citizen's Arrest,Citizen's Arrest,3,C,W,W,5,6,6,6,6,6,7,6.000,7.0,5.0,2.0,0.3
Captain's Call,Captain's Call,4,C,W,W,7,5,6,6,7,6,4,5.857,7.0,4.0,3.0,0.8
Take Up the Shield,Take Up the Shield,2,C,W,W,5,6,4,7,6,7,4,5.571,7.0,4.0,3.0,1.1
Mesa Cavalier,Mesa Cavalier,3,C,W,W,6,5,5,4,6,6,5,5.286,6.0,4.0,2.0,0.6
Battlewing Mystic,Battlewing Mystic,2,U,UR,U,7,8,7,9,8,8,6,7.571,9.0,6.0,3.0,0.8


This is the entire list of cards, ordered by rarity, then by average rating amongst contributers.

In [14]:
ordering = 'mean'
commons = display_frame(frame, order=ordering, filters=[rarity_filter('C')]).data
uncommons = display_frame(frame, order=ordering, filters=[rarity_filter('U')]).data
rare = display_frame(frame, order=ordering, filters=[rarity_filter('R')]).data
mythic = display_frame(frame, order=ordering, filters=[rarity_filter('M')]).data

display = pd.concat([commons, uncommons, rare, mythic])
style_frame(display)

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist
Card,,,,,,,,,,,,,,,,,
Lightning Strike,Lightning Strike,2,C,R,R,9,9,8,10,9,9,7,8.714,10.0,7.0,3.0,0.7
Ghitu Amplifier,Ghitu Amplifier,2,C,UR,R,7,7,6,8,7,7,7,7.000,8.0,6.0,2.0,0.3
Tolarian Geyser,Tolarian Geyser,3,C,WU,U,7,6,6,7,8,7,6,6.714,8.0,6.0,2.0,0.6
Phyrexian Rager,Phyrexian Rager,3,C,B,B,6,6,6,8,6,7,7,6.571,8.0,6.0,2.0,0.7
Argivian Cavalier,Argivian Cavalier,3,C,W,W,7,7,7,8,7,6,4,6.571,8.0,4.0,4.0,0.9
Radiant Grove,Radiant Grove,0,C,WG,,6,6,7,7,7,6,6,6.429,7.0,6.0,1.0,0.5
Tangled Islet,Tangled Islet,0,C,UG,,6,6,7,7,7,6,6,6.429,7.0,6.0,1.0,0.5
Wooded Ridgeline,Wooded Ridgeline,0,C,RG,,6,6,7,7,7,6,6,6.429,7.0,6.0,1.0,0.5
Talas Lookout,Talas Lookout,4,C,U,U,7,6,5,8,7,6,6,6.429,8.0,5.0,3.0,0.8


### Color Ratings

This is a little rougher, and more difficult to parse, but this is the avreage grade for each contirbutor given to cards of 'x' colour and 'y' rarity. it should serve as a rough estimation of how strong each person thinks a given colour is, though doesn't necessarily translate well to colour combinations.

In [15]:
get_avgs(frame)

Zac  Klarm  Phyrre  Nomad  Catharsis  Rewind  Davis  mean   max  min  \
M W  10.0    9.0    10.0   10.0       10.0    11.0   11.0  10.1  11.0  9.0   
  U   7.0    6.5     5.5   10.0        7.5     9.0    5.5   7.3  10.0  5.5   
  B   7.0    9.0     8.0    9.0        8.0     9.5    6.5   8.1   9.5  6.5   
  R   9.5    9.5    10.0    9.5        9.0    10.5    8.5   9.5  10.5  8.5   
  G   5.5    4.0     4.5    5.5        6.0     6.0    7.0   5.5   7.0  4.0   
R W   7.1    7.1     6.9    8.0        6.2     7.2    7.1   7.0   8.0  6.2   
  U   6.0    6.5     6.0    7.5        6.2     6.6    6.4   6.5   7.5  6.0   
  B   5.9    6.9     5.9    6.0        6.2     6.1    6.1   6.2   6.9  5.9   
  R   6.0    7.5     5.6    6.9        6.5     6.0    5.8   6.3   7.5  5.6   
  G   6.6    6.4     7.1    7.8        6.2     8.2    7.5   7.1   8.2  6.2   
U W   5.1    5.7     5.0    5.7        5.3     6.4    4.6   5.4   6.4  4.6   
  U   5.4    5.5     4.3    6.4        5.3     5.6    4.7   5.3   6.4  4.3   
  B   4.0    4.6     4.0    6.1        4.4     5.0    5.0   4.7   6.1  4.0   
  R   4.5    5.3     4.5    6.4        5.1     6.0    5.0   5.3   6.4  4.5   
  G   5.5    5.8     5.8    7.5        5.5     6.9    5.5   6.1   7.5  5.5   
C W   4.8    4.6     4.7    4.2        4.1     4.8    4.2   4.5   4.8  4.1   
  U   5.0    4.8     4.6    5.0        4.9     4.9    4.5   4.8   5.0  4.5   
  B   3.6    4.5     4.4    5.3        3.9     4.6    5.1   4.5   5.3  3.6   
  R   4.6    4.7     4.4    4.9        4.5     4.8    4.6   4.5   4.9  4.4   
  G   3.8    4.7     4.4    4.4        4.1     4.4    4.8   4.2   4.8  3.8   

     range  dist  
M W    2.0  0.47  
  U    4.5  1.33  
  B    3.0  0.87  
  R    2.0  0.43  
  G    3.0  0.71  
R W    1.8  0.34  
  U    1.5  0.36  
  B    1.0  0.24  
  R    1.9  0.54  
  G    2.0  0.61  
U W    1.8  0.46  
  U    2.1  0.47  
  B    2.1  0.54  
  R    1.9  0.56  
  G    2.0  0.66  
C W    0.7  0.27  
  U    0.5  0.16  
  B    1.7  0.44  
  R    0.5  0.17  
  G    1.0  0.31

These are the rankings of all the non-basic lands.
In short:
- Gx taplands are highest rated
- UR tapland is slightly ahead of other non-GX tap lands
- Taplands are ahead of painlands

In [16]:
lands = [
     'Adarkar Wastes',
     'Caves of Koilos',
     'Shivan Reef',
     'Yavimaya Coast',
     'Sulfurous Springs',
     'Karplusan Forest',
     'Plaza of Heroes',
     'Thran Portal',
     'Crystal Grotto',
     'Idyllic Beachfront',
     'Sunlit Marsh',
     'Sacred Peaks',
     'Radiant Grove',
     'Contaminated Aquifer',
     'Molten Tributary',
     'Tangled Islet',
     'Geothermal Bog',
     'Haunted Mire',
     'Wooded Ridgeline',
]
style_frame(frame.loc[lands])

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist
Card,,,,,,,,,,,,,,,,,
Adarkar Wastes,Adarkar Wastes,0,R,WU,,6,5,5,6,4,6,7,5.571,7.0,4.0,3.0,0.8
Caves of Koilos,Caves of Koilos,0,R,WB,,6,5,5,6,4,6,7,5.571,7.0,4.0,3.0,0.8
Shivan Reef,Shivan Reef,0,R,UR,,6,5,5,6,4,6,7,5.571,7.0,4.0,3.0,0.8
Yavimaya Coast,Yavimaya Coast,0,R,UG,,6,5,5,6,4,6,7,5.571,7.0,4.0,3.0,0.8
Sulfurous Springs,Sulfurous Springs,0,R,BR,,6,5,4,6,4,6,7,5.429,7.0,4.0,3.0,0.9
Karplusan Forest,Karplusan Forest,0,R,RG,,6,5,4,6,4,6,7,5.429,7.0,4.0,3.0,0.9
Plaza of Heroes,Plaza of Heroes,0,R,,,2,3,0,4,5,4,1,2.714,5.0,0.0,5.0,1.5
Thran Portal,Thran Portal,0,R,,,0,4,4,0,4,2,7,3.000,7.0,0.0,7.0,2.0
Crystal Grotto,Crystal Grotto,0,C,,,0,1,3,4,5,6,1,2.857,6.0,0.0,6.0,1.9


### Gut-Check

The two selections below are mainly for my benefit (though I can send any contributor a personalized copy if they ask!), to check my opions of cards against what everyone else thinks. Any card I have graded more than a grade higher or lower than average is shown here, to give a more concrete idea of where _I_ disagree, than where _people_ disagree.

In [17]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] > 1].sort_values('diff', ascending=False))

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist,diff
Card,,,,,,,,,,,,,,,,,,
Drag to the Bottom,Drag to the Bottom,4,R,B,B,9,7,3,9,5,6,7,6.571,9.0,3.0,6.0,1.6,2.428571
Defiler of Faith,Defiler of Faith,5,R,W,W,11,10,8,10,5,9,8,8.714,11.0,5.0,6.0,1.5,2.285714
Defiler of Instinct,Defiler of Instinct,4,R,R,R,11,10,10,10,8,7,8,9.143,11.0,7.0,4.0,1.3,1.857143
Love Song of Night and Day,Love Song of Night and Day,3,U,W,W,6,5,4,2,4,6,2,4.143,6.0,2.0,4.0,1.3,1.857143
Sol'Kanar the Tainted,Sol'Kanar the Tainted,5,M,UBR,UBR,10,9,5,9,7,9,9,8.286,10.0,5.0,5.0,1.3,1.714286
Temporal Firestorm,Temporal Firestorm,5,R,WUR,R,8,10,3,10,4,6,5,6.571,10.0,3.0,7.0,2.4,1.428571
Twinferno,Twinferno,2,U,R,R,5,3,4,4,4,5,1,3.714,5.0,1.0,4.0,1.0,1.285714
"Stenn, Paranoid Partisan","Stenn, Paranoid Partisan",2,R,WU,WU,7,6,6,4,4,5,8,5.714,8.0,4.0,4.0,1.2,1.285714
Founding the Third Path,Founding the Third Path,2,U,U,U,4,3,0,1,4,3,4,2.714,4.0,0.0,4.0,1.3,1.285714


In [18]:
play_frame = frame.copy(True)
play_frame['diff'] = frame['Zac'] - frame['mean']
style_frame(play_frame[play_frame['diff'] < -1].sort_values('diff', ascending=True))

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist,diff
Card,,,,,,,,,,,,,,,,,,
Temporary Lockdown,Temporary Lockdown,3,R,W,W,0,3,1,,5,,6,3.000,6.0,0.0,6.0,2.0,-3.000000
Thran Portal,Thran Portal,0,R,,,0,4,4,0,4,2,7,3.000,7.0,0.0,7.0,2.0,-3.000000
Crystal Grotto,Crystal Grotto,0,C,,,0,1,3,4,5,6,1,2.857,6.0,0.0,6.0,1.9,-2.857143
Balduvian Berserker,Balduvian Berserker,3,U,R,R,2,4,3,7,3,7,6,4.571,7.0,2.0,5.0,1.8,-2.571429
Eerie Soultender,Eerie Soultender,3,C,B,B,1,2,5,4,3,4,5,3.429,5.0,1.0,4.0,1.2,-2.428571
"Karn, Living Legacy","Karn, Living Legacy",4,M,,,0,3,4,3,4,0,2,2.286,4.0,0.0,4.0,1.4,-2.285714
Leaf-Crowned Visionary,Leaf-Crowned Visionary,2,R,G,G,3,4,4,7,5,6,8,5.286,8.0,3.0,5.0,1.5,-2.285714
Monstrous War-Leech,Monstrous War-Leech,4,U,UB,B,1,3,2,6,4,3,4,3.286,6.0,1.0,5.0,1.2,-2.285714
Shadow-Rite Priest,Shadow-Rite Priest,2,R,B,B,2,6,3,3,4,6,6,4.286,6.0,2.0,4.0,1.5,-2.285714


### Specific Checks

These are a list of specific cards I wanted to compare, because they either seem interesting, are cards that feel similar-but-different. Feel free to ask me to add certain cards to this list if you're curious

In [19]:
play_frame.loc[['Love Song of Night and Day', 'Dragon Whelp', 'Soaring Drake', 'Mesa Cavalier', 'Leyline Binding', 'Prayer of Binding', 'Deathbloom Gardener', 'Salvaged Manaworker']]

,Image,CMC,Rarity,Identity,To Cast,Zac,Klarm,Phyrre,Nomad,Catharsis,Rewind,Davis,mean,max,min,range,dist,diff
Card,,,,,,,,,,,,,,,,,,
Love Song of Night and Day,Love Song of Night and Day,3,U,W,W,6,5,4,2,4,6,2,4.142857,6.0,2.0,4.0,1.3,1.857143
Dragon Whelp,Dragon Whelp,4,U,R,R,3,6,6,6,5,5,5,5.142857,6.0,3.0,3.0,0.7,-2.142857
Soaring Drake,Soaring Drake,3,C,U,U,5,5,5,6,4,6,4,5.000000,6.0,4.0,2.0,0.6,0.0
Mesa Cavalier,Mesa Cavalier,3,C,W,W,6,5,5,4,6,6,5,5.285714,6.0,4.0,2.0,0.6,0.714286
Leyline Binding,Leyline Binding,6,R,W,W,6,6,8,6,6,7,9,6.857143,9.0,6.0,3.0,1.0,-0.857143
Prayer of Binding,Prayer of Binding,4,U,W,W,7,8,8,8,7,7,7,7.428571,8.0,7.0,1.0,0.5,-0.428571
Deathbloom Gardener,Deathbloom Gardener,3,C,G,G,3,6,6,2,2,3,5,3.857143,6.0,2.0,4.0,1.6,-0.857143
Salvaged Manaworker,Salvaged Manaworker,2,C,,,4,2,4,5,2,6,3,3.714286,6.0,2.0,4.0,1.2,0.285714
